In [ ]:
library(xml2)

In [ ]:
ComXmlFile  = read_xml("data/com.xml")
ContratXmlFile = read_xml("data/contract.xml")
EngiXmlFile = read_xml("data/engi.xml")

In [ ]:
# Chargement des bibliothèques nécessaires
library(xml2)     # Pour lire et manipuler des fichiers XML
library(dplyr)    # Pour manipuler des tableaux de données (tibbles)
library(purrr)    # Pour la fonctions de programmation fonctionnelle  (walk)
library(stringr)  # Pour manipuler les chaînes de caractères
library(tibble)

xmlToTables_exclu <- function(doc,
                        exclude_clas_containing = NULL) {
  tryCatch({
    # --- vérifications initiales ---
    if (!inherits(doc, "xml_document")) {
      stop("L'entrée doit être un objet xml_document")
    }
    
    root <- xml_root(doc)
    if (is.null(root)) {
      warning("Le document XML est vide")
      return(list())
    }

    # --- préparation des motifs d'exclusion ---
    exclude_patterns <- as.character(exclude_clas_containing %||% character(0))
    has_exclusions <- length(exclude_patterns) > 0

    tables <- list()

    parse_obj <- function(node) {
      tryCatch({
        clas <- xml_attr(node, "clas")
        if (is.na(clas) || is.null(clas)) return()

        # Exclusion si le nom de classe contient un motif indésirable
        if (has_exclusions) {
          drop <- any(str_detect(clas, regex(exclude_patterns, ignore_case = TRUE)))
          if (drop) return()
        }

        props <- xml_find_all(node, "./prop[not(*) and not(.//obj)]")
        if (length(props) == 0) return()

        texts <- xml_text(props) %>% trimws()
        names <- xml_attr(props, "nom") %>% str_remove("^\\$")
        
        if (length(texts) != length(names)) {
          warning(sprintf("Incohérence dans %s : %d valeurs / %d noms",
                          clas, length(texts), length(names)))
          return()
        }

        row <- set_names(texts, names)

        if (is.null(tables[[clas]])) tables[[clas]] <<- tibble()
        tables[[clas]] <<- tryCatch(
          bind_rows(tables[[clas]], as_tibble_row(row)),
          error = function(e) {
            warning(sprintf("bind_rows échoue pour %s : %s", clas, e$message))
            tables[[clas]]
          }
        )

        children <- c(xml_find_all(node, "./prop//obj"),
                      xml_find_all(node, "./prop//entr/obj"))
        walk(children, parse_obj)

      }, error = function(e) {
        warning(sprintf("Erreur lors du traitement du nœud : %s", e$message))
      })
    }

    objs <- tryCatch(xml_find_all(root, ".//obj"),
                     error = function(e) xml_nodeset())
    if (length(objs)) walk(objs, parse_obj)

    tables <- map(tables, ~ tryCatch(
      rename_with(., ~ str_replace_all(.x, "[^[:alnum:]_]", "")),
      error = function(e) .
    ))

    return(tables)

  }, error = function(e) {
    warning("Erreur globale :", e$message)
    return(list())
  })
}

In [ ]:
engi_df = xmlToTables_exclu(EngiXmlFile,exclude_clas = c("lay","item",'seg','zone'))
print(engi_df)


In [ ]:
contrat_df = xmlToTables_exclu(ContratXmlFile)
print(contrat_df)


In [ ]:
comm_df = xmlToTables_exclu(ComXmlFile)
print(comm_df)


In [ ]:
conn = DBI::dbConnect(odbc::odbc(), .connection_string = "Driver={ODBC Driver 17 for SQL Server};", 
            Server = "server-sql-dunin-eve.database.windows.net", Database = "AviviaEve_Copy", 
            UID = "accesAviviaData", PWD = "Citric&Private&Designed&Maturity1&Cloning")

projets = tbl(conn,'projetEW') 
projet_df = projets %>% head(1) %>% pull(ProjetXML) %>% as_xml_document() %>% xmlToTables_exclu()



In [ ]:
projet_df

In [ ]:
library(tibble)

labelled_transaction <- tribble(
  ~variable,          ~libellé,
  "aNote",            "Note interne",
  "documSource",      "Document source",
  "estCQ",            "Est CQ",
  "estCQSup",         "CQ supérieur",
  "estImport",        "Importée",
  "estReprise",       "Reprise",
  "dateAcompte1Emis", "Date émission acompte 1",
  "fgfExclu",         "FGF exclu",
  "installExclu",     "Installation exclue",
  "matExclu",         "Matériel exclu",
  "moExclu",          "MO exclue",
  "pourcAcompte1",    "% Acompte 1",
  "pourcAcompte2",    "% Acompte 2",
  "solde",            "Solde dû",
  "transportExclu",   "Transport exclu",
  "appNotifObligs",   "Approbations obligatoires",
  "appNotifs",        "Approbations / Notifications",
  "codeClient2",      "Code client 2",
  "dateValidite",     "Date de validité",
  "delaiPrevExped",   "Délai prévu expédition (j)",
  "disclaimers",      "Disclaimers",
  "emplacementCre",   "Emplacement création",
  "enErreur",         "En erreur",
  "enPSD",            "Prix sur demande",
  "identCour",        "Identifiant courant",
  "nbJourValide",     "Nb jours validité",
  "nomClient",        "Nom du client",
  "projet",           "Projet",
  "promotions",       "Promotions",
  "qteItems",         "Qté d'items",
  "qteItemsCab",      "Qté items CAB",
  "roleProprietaire", "Rôle propriétaire",
  "sorte",            "Sorte",
  "statutCJobsite",   "Statut chantier",
  "statutCred",       "Statut crédit",
  "type",             "Type",
  "typeProcess",      "Type de processus",
  "validCred",        "Validation crédit",
  "ligne",            "Ligne",
  "avertLivraison",   "Avertissement livraison",
  "cleEnMain",        "Clé en main",
  "codePost",         "Code postal",
  "installation",     "Installation",
  "typeClient",       "Type de client",
  "assemblage",       "Assemblage",
  "edgeColle",        "Bordure collée",
  "gamme",            "Gamme",
  "ciClient",         "CI client",
  "cleItem",          "Clé item",
  "courrielCharge",   "Courriel chargé",
  "createurCourriel", "Courriel créateur",
  "createurNom",      "Nom créateur",
  "dateCreation",     "Date de création",
  "division",         "Division",
  "documCree",        "Document créé",
  "memeCoordsJSCF",   "Même coord. chantier / fact.",
  "sauvegarde",       "Sauvegarde",
  "statutJobsite",    "Statut chantier",
  "code",             "Code",
  "codeTaxe",         "Code taxe 1",
  "codeTaxe2",        "Code taxe 2",
  "fuelExtra",        "Fuel extra",
  "paletteNet",       "Palette net",
  "pourcEsc",         "% Escompte",
  "pourcEscSup",      "% Escompte suppl.",
  "sousTotal",        "Sous-total",
  "tauxTaxe",         "Taux taxe 1",
  "tauxTaxe2",        "Taux taxe 2",
  "total",            "Total",
  "totalHT",          "Total HT",
  "totalTx1",         "Total taxe 1",
  "totalTx2",         "Total taxe 2",
  "transpCumul",      "Transport cumulé",
  "transpExtra",      "Transport extra",
  "transpJobSite",    "Transport chantier",
  "transpNet",        "Transport net",
  "avecLivraison",    "Avec livraison",
  "cumulService",     "Cumul service",
  "cumulVolume",      "Cumul volume",
  "pourcEscAutre",    "% Escompte autres",
  "pourcEscCab",      "% Escompte CAB",
  "pourcEscComptoir", "% Escompte comptoir",
  "pourcEscMoulure",  "% Escompte moulure",
  "pourcEscPanel",    "% Escompte panneau",
  "sousTotalAutre",   "Sous-total autre",
  "sousTotalCab",     "Sous-total CAB",
  "sousTotalComptoir","Sous-total comptoir",
  "sousTotalMoulure", "Sous-total moulure",
  "sousTotalPanel",   "Sous-total panneau",
  "totalComptoirBois","Total comptoir bois",
  "totalComptoirGranite","Total comptoir granite",
  "totalComptoirQuartz","Total comptoir quartz",
  "totalComptoirStratifie","Total comptoir stratifié",
  "totalEscAutre",    "Total escompte autre",
  "totalEscComptoir", "Total escompte comptoir",
  "totalEscPanel",    "Total escompte panneau",
  "totalLivraison",   "Total livraison",
  "unit",             "Unité"
)

labelled_sommaireKit <- tribble(
  ~variable,     ~libellé,
  "code",        "Code",
  "no",          "N°",
  "ligne",       "Ligne",
  "pourcEsc",    "% Escompte",
  "pourcEscSup", "% Escompte suppl.",
  "sousTotal",   "Sous-total"
)

labelled_etape <- tribble(
  ~variable,       ~libellé,
  "createur",      "Créateur",
  "dateCreation",  "Date de création",
  "emplacement",   "Emplacement",
  "statut",        "Statut",
  "tempsCreation", "Temps de création"
)

labelled_disclaimer <- tribble(
  ~variable, ~libellé,
  "condition", "Condition",
  "terme",     "Terme"
)

labelled_lumiere <- tribble(
  ~variable, ~libellé,
  "modele",  "Modèle",
  "qte",     "Qté"
)

labelled_projet <- tribble(
  ~variable,            ~libellé,
  "aDocuments",         "Documents associés",
  "client",             "Client",
  "codeRepresentant",   "Code représentant",
  "codeRepresentant2",  "Code représentant 2",
  "coordsFactTxt",      "Adresse facturation",
  "enFusion",           "En fusion",
  "estReprise",         "Reprise",
  "libre",              "Libre",
  "sorte",              "Sorte",
  "ciClient",           "CI client",
  "coordsExpedTxt",     "Adresse expédition",
  "courrielContact",    "Courriel contact",
  "dateCreation",       "Date création",
  "division",           "Division",
  "memeCoordsJSCF",     "Même coord. chantier/fact.",
  "nomContact",         "Nom contact",
  "nomRepresentant",    "Nom représentant",
  "nomRepresentant2",   "Nom représentant 2",
  "posteContact",       "Poste contact",
  "sauvegarde",         "Sauvegarde",
  "statutJobsite",      "Statut chantier",
  "code",               "Code"
)

labelled_client <- tibble::tribble(
  ~variable, ~libellé,
  "langue",  "Langue",
  "monnaie", "Monnaie",
  "code",    "Code client",
  "nom",     "Nom"
)

labelled_AdrCivique <- tibble::tribble(
  ~variable,  ~libellé,
  "codePost", "Code postal",
  "etatProv", "État / Province",
  "ligne1",   "Adresse ligne 1",
  "pays",     "Pays",
  "region",   "Région",
  "ville",    "Ville",
  "usage",    "Usage",
  "code",     "Code",
  "sorte",    "Sorte"
)
# ---- French labels for table `kit` -----------------------------------------
labelled_kit_fr <- tribble(
  ~variable,                        ~libellé,
  "cumulComptoir",                  "Cumul comptoirs",
  "extraComptoir",                  "Extra comptoir",
  "nbComptoir",                     "Nb comptoirs",
  "sousTotal",                      "Sous-total kit",
  "sousTotalCab",                   "Sous-total cabinets",
  "sousTotalEff",                   "Sous-total effectif",
  "sousTotalMoulure",               "Sous-total moulures",
  "sousTotalPanel",                 "Sous-total panneaux",
  "totalComptoir",                  "Total comptoirs",
  "totalComptoirEff",               "Total comptoirs effectif",
  "sousTotalKitEff",                "Sous-total kit effectif",
  "longComptoir",                   "Longueur comptoir",
  "porteModeleB",                   "Modèle porte bas",
  "porteModeleBSensGrain",          "Sens grain porte B",
  "porteModeleB_epais",             "Épaisseur porte B",
  "t_porteModeleB_epais",           "Texte épaisseur porte B",
  "porteModeleFacadeB_epais",       "Épaisseur façade B",
  "t_porteModeleFacadeB_epais",     "Texte épaisseur façade B",
  "porteModeleFacadeH_epais",       "Épaisseur façade H",
  "t_porteModeleFacadeH_epais",     "Texte épaisseur façade H",
  "porteModeleH",                   "Modèle porte haut",
  "porteModeleHSensGrain",          "Sens grain porte H",
  "porteModeleH_epais",             "Épaisseur porte H",
  "t_porteModeleH_epais",           "Texte épaisseur porte H",
  "surfaceComptoir",                "Surface comptoir",
  "moulLumHeight",                  "Hauteur moulure lum.",
  "t_moulLumHeight",                "Texte hauteur moulure lum.",
  "moulLumModele",                  "Modèle moulure lum.",
  "moulPiedHeight",                 "Hauteur moulure pied",
  "t_moulPiedHeight",               "Texte hauteur moulure pied",
  "moulPiedModele",                 "Modèle moulure pied",
  "sousTotalKit",                   "Sous-total kit",
  "typeColor",                      "Type couleur",
  "porteModeleFacadeB",             "Façade bas modèle",
  "porteModeleFacadeH",             "Façade haut modèle",
  "cle",                            "Clé",
  "code",                           "Code kit",
  "enErreur",                       "En erreur",
  "no",                             "N°",
  "nom",                            "Nom kit",
  "qte",                            "Quantité",
  "noKitItem",                      "N° item kit",
  "cost_matCumul",                  "Coût mat. cumul.",
  "cost_moCumul",                   "Coût MO cumul.",
  "price_matCumul",                 "Prix mat. cumul.",
  "price_moCumul",                  "Prix MO cumul.",
  "statErr",                        "Statut erreur",
  "pourcEsc",                       "% Escompte",
  "pourcEscSup",                    "% Escompte suppl.",
  "assemblage",                     "Assemblage",
  "bd_edgePorte",                   "Bordure porte BD",
  "bd_finishColor",                 "Couleur fini BD",
  "bd_handle",                      "Poignée BD",
  "bd_handleOri",                   "Orientation poignée BD",
  "bd_serie",                       "Série BD",
  "bd_specie",                      "Essence BD",
  "cabAlignBase",                   "Alignement base meuble",
  "t_cabAlignBase",                 "Texte alignement base",
  "cabConstruct",                   "Construction meuble",
  "cabDegagementWall",              "Dégagement mur meuble",
  "t_cabDegagementWall",            "Texte dégagement mur",
  "cabDesign",                      "Design meuble",
  "cabHautComptoir",                "Hauteur comptoir meuble",
  "t_cabHautComptoir",              "Texte hauteur comptoir",
  "cabHinge",                       "Charnière meuble",
  "cabTallHeightAlign",             "Alignement haut meuble haut",
  "t_cabTallHeightAlign",           "Texte alignement haut",
  "cabTypeComptoir",                "Type comptoir meuble",
  "cabWallProf",                    "Profondeur mur meuble",
  "t_cabWallProf",                  "Texte prof. mur",
  "comptoirGpCouleur",              "Groupe couleur comptoir",
  "comptoirMod",                    "Modèle comptoir",
  "detailSection",                  "Détail section",
  "df_edgePorte",                   "Bordure porte DF",
  "df_finishColor",                 "Couleur fini DF",
  "df_handle",                      "Poignée DF",
  "df_handleOri",                   "Orientation poignée DF",
  "df_serie",                       "Série DF",
  "df_specie",                      "Essence DF",
  "drawerConst",                    "Construction tiroir",
  "drawerSlide",                    "Glissière tiroir",
  "edgeBanding",                    "Rubane de chant",
  "edgeColle",                      "Joint de chant",
  "edgePorte",                      "Bordure porte",
  "edgeTablette",                   "Bordure tablette",
  "equerre",                        "Équerre",
  "kitUtilisation",                 "Utilisation kit",
  "ligne",                          "Ligne",
  "matPatte",                       "Matériau patte",
  "patteAchetee",                   "Patte achetée",
  "serie",                          "Série",
  "signature",                      "Signature",
  "specie",                         "Essence",
  "tabletteType",                   "Type tablette",
  "typeKit",                        "Type kit",
  "wd_edgeBanding",                 "Rubane chant WD",
  "df_edgeBanding",                 "Rubane chant DF",
  "cabToeKickHeight",               "Hauteur plinthe meuble",
  "t_cabToeKickHeight",             "Texte hauteur plinthe",
  "cabToeKickProf",                 "Profondeur plinthe meuble",
  "t_cabToeKickProf",               "Texte prof. plinthe",
  "cabToeKickType",                 "Type plinthe meuble",
  "sousTotalAutre",                 "Sous-total autres"
)



In [ ]:
library(labelled)
library(dplyr)
set_labels <- function(df,
                       dict,
                       var_col  = "variable",
                       label_col = "libellé") {
  # Conserver uniquement les labels dont la variable existe
  labels_to_set <- dict %>%
    filter(.data[[var_col]] %in% names(df)) %>%
    deframe()
  
  # Appliquer les labels
  df %>%
    set_variable_labels(.labels = labels_to_set)
}


In [ ]:
set_labels(comm_df$Comm,labelled_transaction)
set_labels(contrat_df$SommaireKit,labelled_sommaireKit)
set_labels(comm_df$Etape,labelled_etape)
set_labels(contrat_df$Disclaimer,labelled_disclaimer)
set_labels(contrat_df$Lumiere,labelled_lumiere)
set_labels(projet_df$Projet,labelled_projet)
set_labels(projet_df$Client,labelled_client)
set_labels(projet_df$AdrCivique,labelled_AdrCivique)
set_labels(engi_df$Kit,labelled_kit_fr)


In [ ]:
library(DT)
library(labelled)
library(dplyr)

#' Affiche les champs et les labels d'un tibble sous forme de DataTable
#'

show_dict <- function(df) {
  # Récupérer les noms des colonnes et leurs labels
  var_labels <- var_label(df)
  
  # Créer un tibble pour afficher les noms des colonnes et les labels
  dict_df <- tibble(
    Champ = names(df),
    Label = var_labels
  )
  
  # Afficher le tableau sous forme de DataTable interactive
  datatable(dict_df, 
            options = list(pageLength = 5), 
            class = "compact stripe")
}



In [ ]:
show_dict(comm_df$Comm)
show_dict(comm_df$Etape)